In [18]:
import torch
import numpy as np

input_size = 4
hidden_size = 2

h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

# shpae = (3,5,4) --> (batch, seqence length, input_size)
input_data_np = np.array([[h,e,l,l,o],
                         [e,o,l,l,l],
                         [l,l,e,e,l]], dtype = np.float32)
input_data = torch.Tensor(input_data_np)

rnn = torch.nn.RNN(input_size, hidden_size)

# shape = (3,5,2) --> (batch, seqence, out)
outputs, _status = rnn(input_data)

In [19]:
print("input shpae = ", input_data.shape)
print("hidden state's shape = ",_status.shape)
print("output shape = ", outputs.shape)

input shpae =  torch.Size([3, 5, 4])
hidden state's shape =  torch.Size([1, 5, 2])
output shape =  torch.Size([3, 5, 2])


# hi hello를 예측해보자

In [140]:
# data 준비
char_set = ['h','i','e','l','o']

input_size = len(char_set) #5
hidden_size = len(char_set) #5
learning_rate = 0.1



x_data = [[0,1,0,2,3,3]] # hi hell 까지가 Input
x_one_hot = [[[1,0,0,0,0],
              [0,1,0,0,0],
              [1,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,1,0],
              [0,0,0,1,0]]]
y_data = [[1,0,2,3,3,4]]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)





In [141]:
X.shape # (batch, sequence, input ) = (1, 6, 5)


torch.Size([1, 6, 5])

In [142]:
# 모델링
rnn = torch.nn.RNN(input_size, hidden_size, batch_first = True)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = learning_rate)

#학습
for i in range(10):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1,input_size), Y.view(-1))
    
    loss.backward()
    optimizer.step()
    
    result = outputs.data.numpy().argmax(axis = 2)
    result_str = "".join([char_set[c] for c in np.squeeze(result)])
    
    print(i, "loss", loss.item(), "prediciton:", result, "True Y:", y_data, "pred str:", result_str)

0 loss 1.6162933111190796 prediciton: [[1 3 3 3 3 3]] True Y: [[1, 0, 2, 3, 3, 4]] pred str: illlll
1 loss 1.410448670387268 prediciton: [[1 3 3 3 3 3]] True Y: [[1, 0, 2, 3, 3, 4]] pred str: illlll
2 loss 1.2946723699569702 prediciton: [[2 2 2 3 3 3]] True Y: [[1, 0, 2, 3, 3, 4]] pred str: eeelll
3 loss 1.1827173233032227 prediciton: [[2 0 2 3 3 3]] True Y: [[1, 0, 2, 3, 3, 4]] pred str: ehelll
4 loss 1.0682591199874878 prediciton: [[2 0 2 3 3 4]] True Y: [[1, 0, 2, 3, 3, 4]] pred str: ehello
5 loss 0.9746897220611572 prediciton: [[1 0 2 3 3 4]] True Y: [[1, 0, 2, 3, 3, 4]] pred str: ihello
6 loss 0.9053593277931213 prediciton: [[1 0 2 3 3 4]] True Y: [[1, 0, 2, 3, 3, 4]] pred str: ihello
7 loss 0.8388474583625793 prediciton: [[1 0 2 3 3 4]] True Y: [[1, 0, 2, 3, 3, 4]] pred str: ihello
8 loss 0.7758391499519348 prediciton: [[1 0 2 3 3 4]] True Y: [[1, 0, 2, 3, 3, 4]] pred str: ihello
9 loss 0.7263551354408264 prediciton: [[1 0 2 3 3 4]] True Y: [[1, 0, 2, 3, 3, 4]] pred str: ihello


In [172]:
smaple = "if you want you"
char_set = list(set(sample))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

# hyper parameter
dic_size = len(char_dic)
hidden_size = len(char_dic) 
learning_rate = 0.1

#data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

{'f': 0, ' ': 1, 'o': 2, 'i': 3, 'w': 4, 'y': 5, 'a': 6, 't': 7, 'n': 8, 'u': 9}


In [179]:
x_data.shape

AttributeError: 'list' object has no attribute 'shape'

In [173]:
print("dic_size = ", dic_size, "hidden_size = ", hidden_size)

# batch, sequence length, input_size
print("X.shape = ", X.shape, "Y.shape = ", Y.shape)

dic_size =  10 hidden_size =  10
X.shape =  torch.Size([1, 14, 10]) Y.shape =  torch.Size([1, 14])


In [175]:
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True) # hidden size는 여기서 처음 사용
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), learning_rate)

In [176]:
for i in range(10):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  2.2262415885925293 prediction:  [[1 1 1 1 1 2 1 2 1 8 1 1 1 1]] true Y:  [[0, 1, 5, 2, 9, 1, 4, 6, 8, 7, 1, 5, 2, 9]] prediction str:       o o n    
1 loss:  1.9128385782241821 prediction:  [[1 1 2 2 2 2 2 2 2 1 1 2 2 2]] true Y:  [[0, 1, 5, 2, 9, 1, 4, 6, 8, 7, 1, 5, 2, 9]] prediction str:    ooooooo  ooo
2 loss:  1.7039344310760498 prediction:  [[1 1 5 2 9 5 2 6 6 9 1 5 6 9]] true Y:  [[0, 1, 5, 2, 9, 1, 4, 6, 8, 7, 1, 5, 2, 9]] prediction str:    youyoaau yau
3 loss:  1.5387998819351196 prediction:  [[1 1 5 6 9 1 5 6 8 1 1 5 6 9]] true Y:  [[0, 1, 5, 2, 9, 1, 4, 6, 8, 7, 1, 5, 2, 9]] prediction str:    yau yan  yau
4 loss:  1.4897459745407104 prediction:  [[0 1 5 6 9 1 5 5 8 1 1 5 6 0]] true Y:  [[0, 1, 5, 2, 9, 1, 4, 6, 8, 7, 1, 5, 2, 9]] prediction str:  f yau yyn  yaf
5 loss:  1.4072916507720947 prediction:  [[0 1 5 6 9 1 5 6 8 1 1 5 6 9]] true Y:  [[0, 1, 5, 2, 9, 1, 4, 6, 8, 7, 1, 5, 2, 9]] prediction str:  f yau yan  yau
6 loss:  1.3524926900863647 prediction:  [[0 1

In [180]:
outputs.shape

torch.Size([1, 14, 10])

In [182]:
outputs.view(-1, dic_size).shape

torch.Size([14, 10])

In [183]:
result.shape

(1, 14)